In [20]:
# COM MACD
from datetime import datetime
import MetaTrader5 as mt5
import time
import telepot
import pytz

agora = datetime.now()
print(f'Buscando dados...{agora}')
# importamos o módulo pandas para exibir os dados recebidos na forma de uma tabela
import pandas as pd
pd.set_option('display.max_columns', 500) # número de colunas
pd.set_option('display.width', 1500)      # largura máxima da tabela
pd.options.mode.chained_assignment = None  # default='warn'


if not mt5.initialize(login=54679378, server="MetaQuotes-Demo", password="hz7ulfri"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

symbol = ["EURUSD","GBPUSD","USDJPY","USDCHF","USDCAD","AUDUSD","NZDUSD"]
#item = "WDOZ21"

for item in symbol:
    print(f'Ativo: {item}')
    timezone = pytz.timezone("Etc/UTC")
    utc_from = datetime(2021, 11, 30, tzinfo=timezone)
    rates = mt5.copy_rates_from(item, mt5.TIMEFRAME_M5, utc_from, 113)
    rates_frame = pd.DataFrame(rates)
    rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
    resumo = rates_frame[['time','open','close','spread']]
    #resumo

    #Criar Sinais de compra e venda
    openMME = resumo['open'].ewm(span=3).mean()      #cor: VERDE(3)
    closeMME = resumo['close'].ewm(span=30).mean()   #cor: VERMELHA(30)
    resumo.insert(loc=4,column='MMEopen',value=openMME)
    resumo.insert(loc=5,column='MMEclose',value=closeMME)
    #print(resumo.tail())

    # MACD
    resumo['EMA12'] = resumo.close.ewm(span=12).mean()
    resumo['EMA26'] = resumo.close.ewm(span=26).mean()
    resumo['MACD'] = resumo.EMA12 - resumo.EMA26
    resumo['signal'] = resumo.MACD.ewm(span=9).mean()
    resumo['histog'] = resumo['MACD'] - resumo['signal']
    #display(resumo.tail(60))
    #teste = resumo[['MACD','signal','histog']]
    #display(teste.tail(60))

    # MONITORAMENTO
    pd.options.mode.chained_assignment = None
    resumo['flag'] = ''
    resumo['sinal'] = ''

    for i in range (1, len(resumo)):
        if resumo['MMEopen'][i] > resumo['MMEclose'][i]:
            resumo['flag'][i] = 'COMPRA'
        else:
            resumo['flag'][i] = 'VENDA'

    for x in range(1,len(resumo)):
        if resumo['flag'][x] == resumo['flag'][x-1]:
            resumo['sinal'][x] = ''
        else:
            resumo['sinal'][x] = 'sinal'

    flag = resumo['flag'].iloc[-1]

    #resumo['sinal'].iloc[-1] = 'sinal'
    #resumo['flag'].iloc[-1] = 'COMPRA'

    if resumo['sinal'].iloc[-1] == 'sinal':
        bot = telepot.Bot('1852343442:AAEBBS1NjjFRIqt-XTbb3rzRxipvk8ZqI5I')
        bot.sendMessage(-351556985, f'ATENÇÃO! MUDANÇA DE STATUS: >> {item} - {flag} <<')
        print('Dados encontrados e enviados via Telegram'.upper())


    # AVISO DE SAÍDA DA OPERAÇÃO    
    compraHISTOG = resumo['histog'].iloc[-4] > 0 and resumo['histog'].iloc[-3] > 0 and resumo['histog'].iloc[-2] > 0
    compraSIGNAL = resumo['signal'].iloc[-4] > 0 and resumo['signal'].iloc[-3] > 0 and resumo['signal'].iloc[-2] > 0    
    lucroCOMPRA = compraHISTOG and resumo['histog'].iloc[-1] < 0 and compraSIGNAL and resumo['signal'].iloc[-1] > 0
    resumo['lucroCOMPRA'] = lucroCOMPRA
    #lucroCOMPRA = True

    vendaHISTOG = resumo['histog'].iloc[-4] < 0 and resumo['histog'].iloc[-3] < 0 and resumo['histog'].iloc[-2] < 0
    vendaSIGNAL = resumo['signal'].iloc[-4] < 0 and resumo['signal'].iloc[-3] < 0 and resumo['signal'].iloc[-2] < 0     
    lucroVENDA = compraHISTOG and resumo['histog'].iloc[-1] > 0 and compraSIGNAL and resumo['signal'].iloc[-1] < 0
    resumo['lucroVENDA'] = lucroVENDA
    #lucroVENDA = True

    if lucroCOMPRA == True:
        bot = telepot.Bot('1852343442:AAEBBS1NjjFRIqt-XTbb3rzRxipvk8ZqI5I')
        bot.sendMessage(-351556985, f'ATENÇÃO! LUCRO MÁXIMO NA COMPRA: >> {item} <<')
        print('Dados encontrados e enviados via Telegram'.upper())

    if lucroVENDA == True:
        bot = telepot.Bot('1852343442:AAEBBS1NjjFRIqt-XTbb3rzRxipvk8ZqI5I')
        bot.sendMessage(-351556985, f'ATENÇÃO! LUCRO MÁXIMO NA VENDA: >> {item} <<')
        print('Dados encontrados e enviados via Telegram'.upper())


    display(resumo.tail(3))

    time.sleep(3)

print('')
print('Script executado com sucesso!'.upper())
#time.sleep()
    

Buscando dados...2021-11-29 18:02:08.281158
Ativo: EURUSD


,time,open,close,spread,MMEopen,MMEclose,EMA12,EMA26,MACD,signal,histog,flag,sinal,lucroCOMPRA,lucroVENDA
110,2021-11-29 22:50:00,1.12788,1.12773,0,1.127932,1.127481,1.127799,1.127537,0.000262,0.000263,-7.012647e-07,COMPRA,,False,False
111,2021-11-29 22:55:00,1.12773,1.12792,0,1.127831,1.127510,1.127817,1.127565,0.000252,0.000261,-8.356144e-06,COMPRA,,False,False
112,2021-11-29 23:00:00,1.12792,1.12818,1,1.127875,1.127553,1.127873,1.127611,0.000262,0.000261,1.498389e-06,COMPRA,,False,False


Ativo: GBPUSD


,time,open,close,spread,MMEopen,MMEclose,EMA12,EMA26,MACD,signal,histog,flag,sinal,lucroCOMPRA,lucroVENDA
110,2021-11-29 22:50:00,1.33007,1.33004,5,1.330118,1.329800,1.330003,1.329815,0.000188,0.000128,0.000060,COMPRA,,False,False
111,2021-11-29 22:55:00,1.33004,1.33023,6,1.330079,1.329828,1.330038,1.329845,0.000192,0.000141,0.000051,COMPRA,,False,False
112,2021-11-29 23:00:00,1.33022,1.33038,8,1.330150,1.329864,1.330090,1.329885,0.000205,0.000154,0.000051,COMPRA,,False,False


Ativo: USDJPY


...

Ativo: AUDUSD


,time,open,close,spread,MMEopen,MMEclose,EMA12,EMA26,MACD,signal,histog,flag,sinal,lucroCOMPRA,lucroVENDA
110,2021-11-29 22:50:00,0.71306,0.71305,3,0.713054,0.712961,0.713026,0.712973,0.000054,0.000051,0.000003,COMPRA,,False,False
111,2021-11-29 22:55:00,0.71305,0.71306,5,0.713052,0.712967,0.713032,0.712979,0.000052,0.000051,0.000001,COMPRA,,False,False
112,2021-11-29 23:00:00,0.71305,0.71317,8,0.713051,0.712980,0.713053,0.712993,0.000060,0.000053,0.000007,COMPRA,,False,False


Ativo: NZDUSD


,time,open,close,spread,MMEopen,MMEclose,EMA12,EMA26,MACD,signal,histog,flag,sinal,lucroCOMPRA,lucroVENDA
110,2021-11-29 22:50:00,0.68099,0.68095,7,0.680960,0.680541,0.680786,0.680582,0.000204,0.000167,0.000037,COMPRA,,False,False
111,2021-11-29 22:55:00,0.68095,0.68084,7,0.680955,0.680560,0.680794,0.680601,0.000193,0.000172,0.000021,COMPRA,,False,False
112,2021-11-29 23:00:00,0.68083,0.68076,9,0.680893,0.680573,0.680789,0.680613,0.000176,0.000173,0.000003,COMPRA,,False,False



SCRIPT EXECUTADO COM SUCESSO!


In [ ]:
# RELATÓRIO DAS POSIÇÕES ABERTAS
info_posicoes = mt5.positions_get()
print(info_posicoes)
df = pd.DataFrame(list(info_posicoes), columns=info_posicoes[0]._asdict().keys())
display(df)

In [ ]:
# aviso de saída: NACIONAIS

# COM MACD
from datetime import datetime
import MetaTrader5 as mt5
import time
import telepot


agora = datetime.now()
print(f'Buscando dados...{agora}')
# importamos o módulo pandas para exibir os dados recebidos na forma de uma tabela
import pandas as pd
pd.set_option('display.max_columns', 500) # número de colunas
pd.set_option('display.width', 1500)      # largura máxima da tabela
pd.options.mode.chained_assignment = None  # default='warn'
import pytz

#if not mt5.initialize(login=50717088, server="ICMarketsSC-Demo", password="2DLJdDev"):
#if not mt5.initialize(login=54679378, server="MetaQuotes-Demo", password="hz7ulfri"):
if not mt5.initialize(login=1092947504, server="ClearInvestimentos-DEMO", password="Joh0516"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()



ativos = ["WDOZ21","WDOZ21"]

for item in ativos:
    # CRIAÇÃO DOS CÁLCULOS (MÉDIAS)
    # definimos o fuso horário como UTC
    timezone = pytz.timezone("Etc/UTC")
    # criamos o objeto datetime no fuso horário UTC para que não seja aplicado o deslocamento do fuso horário local
    utc_from = datetime(2021, 11, 30, tzinfo=timezone)
    # recebemos 10 barras de EURUSD H4 a partir de 01/10/2019 no fuso horário UTC
    rates = mt5.copy_rates_from(item, mt5.TIMEFRAME_M5, utc_from, 113)
    # a partir dos dados recebidos criamos o DataFrame
    rates_frame = pd.DataFrame(rates)
    # convertemos o tempo em segundos no formato datetime
    rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
    resumo = rates_frame[['time','open','close','spread']]
    #resumo

    #Criar Sinais de compra e venda
    openMME = resumo['open'].ewm(span=3).mean()      #cor: VERDE(3)
    closeMME = resumo['close'].ewm(span=30).mean()   #cor: VERMELHA(30)
    resumo.insert(loc=4,column='MMEopen',value=openMME)
    resumo.insert(loc=5,column='MMEclose',value=closeMME)
    #print(resumo.tail())

    # MACD
    resumo['EMA12'] = resumo.close.ewm(span=12).mean()
    resumo['EMA26'] = resumo.close.ewm(span=26).mean()
    resumo['MACD'] = resumo.EMA12 - resumo.EMA26
    resumo['signal'] = resumo.MACD.ewm(span=9).mean()
    resumo['histog'] = resumo['MACD'] - resumo['signal']
    #display(resumo.tail(60))

    #teste = resumo[['MACD','signal','histog']]
    #display(teste.tail(60))

    # MONITORAMENTO
    pd.options.mode.chained_assignment = None
    resumo['flag'] = ''
    resumo['sinal'] = ''

    for i in range (1, len(resumo)):
        if resumo['MMEopen'][i] > resumo['MMEclose'][i]:
            resumo['flag'][i] = 'COMPRA'
        else:
            resumo['flag'][i] = 'VENDA'

    for x in range(1,len(resumo)):
        if resumo['flag'][x] == resumo['flag'][x-1]:
            resumo['sinal'][x] = ''
        else:
            resumo['sinal'][x] = 'sinal'


    # RELATÓRIO DAS POSIÇÕES
    info_posicoes = mt5.positions_get()
    if info_posicoes:
        #print(info_posicoes)
        df = pd.DataFrame(list(info_posicoes), columns=info_posicoes[0]._asdict().keys())
        display(df)
        ticket = df['ticket'].iloc[0]
        natureza = df['type'].iloc[0]

    flag = resumo['flag'].iloc[-1]

    #resumo['sinal'].iloc[-1] = 'sinal'
    #resumo['flag'].iloc[-1] = 'COMPRA'


    # AVISO DE SAÍDA DA OPERAÇÃO    
    compraHISTOG = resumo['histog'].iloc[-4] > 0 and resumo['histog'].iloc[-3] > 0 and resumo['histog'].iloc[-2] > 0
    compraSIGNAL = resumo['signal'].iloc[-4] > 0 and resumo['signal'].iloc[-3] > 0 and resumo['signal'].iloc[-2] > 0    
    lucroCOMPRA = compraHISTOG and resumo['histog'].iloc[-1] < 0 and compraSIGNAL and resumo['signal'].iloc[-1] > 0

    vendaHISTOG = resumo['histog'].iloc[-4] < 0 and resumo['histog'].iloc[-3] < 0 and resumo['histog'].iloc[-2] < 0
    vendaSIGNAL = resumo['signal'].iloc[-4] < 0 and resumo['signal'].iloc[-3] < 0 and resumo['signal'].iloc[-2] < 0     
    lucroVENDA = compraHISTOG and resumo['histog'].iloc[-1] > 0 and compraSIGNAL and resumo['signal'].iloc[-1] < 0


    if lucroCOMPRA == True:
        bot = telepot.Bot('1852343442:AAEBBS1NjjFRIqt-XTbb3rzRxipvk8ZqI5I')
        bot.sendMessage(-351556985, f'ATENÇÃO! LUCRO MÁXIMO NA COMPRA: >> {item} - {flag} <<')
        print('Dados encontrados e enviados via Telegram'.upper())

    if lucroVENDA == True:
        bot = telepot.Bot('1852343442:AAEBBS1NjjFRIqt-XTbb3rzRxipvk8ZqI5I')
        bot.sendMessage(-351556985, f'ATENÇÃO! LUCRO MÁXIMO NA VENDA: >> {item} - {flag} <<')
        print('Dados encontrados e enviados via Telegram'.upper())


    display(resumo.tail(3))

    time.sleep(2)

print('')
print('Script executado com sucesso. Fechando o programa...'.upper())
#time.sleep()